In [2]:
import warnings

import numpy as np
import pandas as pd

from jre_utils.datapath import factor_data_paths

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [3]:
new_dwellings_processed_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
new_dwellings_unprocessed_path = factor_data_paths["unprocessed"]["new_dwellings"]["municipality"]

existing_dwellings_processed_path = factor_data_paths["processed"]["existing_dwellings"]["municipality"]
existing_dwellings_unprocessed_path = factor_data_paths["unprocessed"]["existing_dwellings"]["municipality"]

In [4]:
new_dwellings_df = pd.read_csv(new_dwellings_unprocessed_path)
existing_dwellings_df = pd.read_csv(existing_dwellings_unprocessed_path)

In [5]:
new_dwellings_df = pd.read_csv(new_dwellings_unprocessed_path)
existing_dwellings_df = pd.read_csv(existing_dwellings_unprocessed_path)

existing_dwellings_df = existing_dwellings_df[existing_dwellings_df["year"] == 2018]
existing_dwellings_df = existing_dwellings_df.drop_duplicates(subset=["year", "area_code"])
existing_dwellings_df = existing_dwellings_df[existing_dwellings_df["existing_dwellings"].apply(lambda x: x.isdigit())]
existing_dwellings_df["existing_dwellings"] = existing_dwellings_df["existing_dwellings"].astype(float)

new_dwellings_df = new_dwellings_df.drop_duplicates(subset=["year", "area_code"])
new_dwellings_df = new_dwellings_df[new_dwellings_df["new_dwellings"].apply(lambda x: x.isdigit())]
new_dwellings_df["new_dwellings"] = new_dwellings_df["new_dwellings"].astype(float)

new_dwellings_df = new_dwellings_df.merge(existing_dwellings_df, on=["year", "area_code"], suffixes=["", "_y"], how="left")

new_dwellings_df["del_dwellings_count"] = new_dwellings_df["new_dwellings"]
new_dwellings_df.loc[new_dwellings_df["year"] == 2018, "del_dwellings_count"] = 0

new_dwellings_df["existing_dwellings"] = new_dwellings_df.groupby("area_code")["existing_dwellings"].ffill()
new_dwellings_df["existing_dwellings"] = new_dwellings_df.groupby("area_code")["existing_dwellings"].bfill()

new_dwellings_df_pre = new_dwellings_df[new_dwellings_df["year"] <= 2018]
new_dwellings_df_pre = new_dwellings_df_pre.sort_values(by=["year", "area_code"], ascending=[False, True])
new_dwellings_df_pre["cumulative_del_dwellings_count"] = new_dwellings_df_pre.groupby("area_code")["del_dwellings_count"].cumsum()
new_dwellings_df_pre["existing_dwellings"] = new_dwellings_df_pre["existing_dwellings"] - new_dwellings_df_pre["cumulative_del_dwellings_count"]

new_dwellings_df_post= new_dwellings_df[new_dwellings_df["year"] > 2018]
new_dwellings_df_post = new_dwellings_df_post.sort_values(by=["year", "area_code"], ascending=[True, True])
new_dwellings_df_post["cumulative_del_dwellings_count"] = new_dwellings_df_post.groupby("area_code")["new_dwellings"].cumsum()
new_dwellings_df_post["existing_dwellings"] = new_dwellings_df_post["existing_dwellings"] + new_dwellings_df_post["cumulative_del_dwellings_count"]

new_dwellings_df = pd.concat([new_dwellings_df_pre, new_dwellings_df_post], ignore_index=True).sort_values(by=["year", "area_code"], ascending=[False, True])

new_dwellings_df["new_dwellings_ratio"] = new_dwellings_df["new_dwellings"] / new_dwellings_df["existing_dwellings"]

new_dwellings_df = new_dwellings_df.drop(columns=["del_dwellings_count", "cumulative_del_dwellings_count", "area_y"])

In [6]:
log_normalize_columns = [
    "new_dwellings",
    "existing_dwellings",
    "new_dwellings_ratio",
]

normalize_columns = [
    "new_dwellings_ratio",
]

for column in log_normalize_columns:
    new_dwellings_df[f"{column}_log"] = new_dwellings_df[column].apply(
        lambda x: np.log10(1 + x)
    )
    new_dwellings_df[f"{column}_log_normalized_yearly"] = new_dwellings_df.groupby(
        "year"
    )[f"{column}_log"].transform(lambda x: (x - x.mean()) / x.std())

for column in normalize_columns:
    new_dwellings_df[f"{column}_normalized_yearly"] = new_dwellings_df.groupby("year")[
        column
    ].transform(lambda x: (x - x.mean()) / x.std())

new_dwellings_df.to_csv(new_dwellings_processed_path, index=False)

In [41]:
# I have a field in a pandas dataframe that is df["existing_dwellings"]
# The existing dwellings are given at 5 year intervals for each area
# I will interpolate the values for the years in between
# For example, is area X has 100 dwellings in 2010 and 200 dwellings in 2015, I will interpolate the values for 2011, 2012, 2013, 2014
# The values for 2011, 2012, 2013, 2014 will be 120, 140, 160, 180 respectively
# Give me the code to do the above 

